## Important libraries

In [1]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
import imutils
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
import os

## import of dataset and labelling

In [2]:
import os
import cv2
import numpy as np

# Folder paths for images with masks and without masks
mask_folder = r'C:\Users\DELL\Face mask detection\with_mask'
without_mask_folder = r'C:\Users\DELL\Face mask detection\without_mask'

# Initialize empty lists to store image data and labels
images = []
labels = []

# Import images with masks
for filename in os.listdir(mask_folder):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        img_path = os.path.join(mask_folder, filename)
        try:
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
            img = cv2.resize(img, (224, 224))  # Resize the image to a fixed size
            images.append(img)
            labels.append(1)  # 1 for images with masks
        except Exception as e:
            print(f"Error occurred while processing {img_path}: {e}")

# Import images without masks
for filename in os.listdir(without_mask_folder):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        img_path = os.path.join(without_mask_folder, filename)
        try:
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
            img = cv2.resize(img, (224, 224))  # Resize the image to a fixed size
            images.append(img)
            labels.append(0)  # 0 for images without masks
        except Exception as e:
            print(f"Error occurred while processing {img_path}: {e}")

# Convert the lists to NumPy arrays for further processing
images = np.array(images, dtype=object)  # Specify dtype=object to handle varying image dimensions
labels = np.array(labels)

# Print the number of imported images
print("Number of images with masks:", np.sum(labels == 1))
print("Number of images without masks:", np.sum(labels == 0))

Number of images with masks: 3724
Number of images without masks: 3828


## Splitting in test train set

In [3]:
from sklearn.model_selection import train_test_split
# Perform train-test split
test_size = 0.2  # Define the ratio of the test set
random_state = 42  # Set the random state for reproducibility
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=test_size, random_state=random_state)

# Print the number of samples in each set
print("Number of samples in training set:", len(X_train))
print("Number of samples in test set:", len(X_test))

Number of samples in training set: 6041
Number of samples in test set: 1511


## Training using CNN Ml model

In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers

# Folder paths for images with masks and without masks
mask_folder = r'C:\Users\DELL\Face mask detection\with_mask'
without_mask_folder = r'C:\Users\DELL\Face mask detection\without_mask'

# Initialize empty lists to store preprocessed image data and labels
preprocessed_images = []
labels = []

# Preprocessing parameters
image_size = (224, 224)  # Target image size for resizing
mean = (0, 0, 0)  # Mean values for image normalization
std = (1, 1, 1)  # Standard deviation values for image normalization

# Import images with masks
for filename in os.listdir(mask_folder):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        img_path = os.path.join(mask_folder, filename)
        try:
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
            img = cv2.resize(img, image_size)  # Resize the image to the target size
            img = img.astype(np.float32)  # Convert image to float32 data type
            img = (img - mean) / std  # Normalize the image
            preprocessed_images.append(img)
            labels.append(1)  # 1 for images with masks
        except Exception as e:
            print(f"Error occurred while processing {img_path}: {e}")

# Import images without masks
for filename in os.listdir(without_mask_folder):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        img_path = os.path.join(without_mask_folder, filename)
        try:
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
            img = cv2.resize(img, image_size)  # Resize the image to the target size
            img = img.astype(np.float32)  # Convert image to float32 data type
            img = (img - mean) / std  # Normalize the image
            preprocessed_images.append(img)
            labels.append(0)  # 0 for images without masks
        except Exception as e:
            print(f"Error occurred while processing {img_path}: {e}")

# Convert the lists to NumPy arrays for further processing
preprocessed_images = np.array(preprocessed_images)
labels = np.array(labels)

# Perform train-test split
test_size = 0.2  # Define the ratio of the test set
random_state = 42  # Set the random state for reproducibility
X_train, X_test, y_train, y_test = train_test_split(preprocessed_images, labels, test_size=test_size, random_state=random_state)

# Define the CNN model architecture
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
batch_size = 16
epochs = 10
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test))

## Evaluate

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Make predictions on the test set
y_pred = model.predict(X_test)
y_pred_classes = np.round(y_pred).flatten()

# Compute evaluation metrics
print("\nClassification Report:")
print(classification_report(y_test, y_pred_classes))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_classes))

## use camera to implement model

In [ ]:
import cv2
import numpy as np
from tensorflow import keras

# Load the trained model
model = keras.models.load_model('path/to/model.h5')

# Define class labels
class_labels = ['Without Mask', 'With Mask']

# Define a function to preprocess the frame
def preprocess_frame(frame):
    # Preprocess the frame (resize, normalize, etc.)
    preprocessed_frame = cv2.resize(frame, (224, 224))
    preprocessed_frame = preprocessed_frame.astype(np.float32)
    preprocessed_frame = (preprocessed_frame - (0, 0, 0)) / (1, 1, 1)  # Normalize based on your preprocessing parameters
    preprocessed_frame = np.expand_dims(preprocessed_frame, axis=0)
    return preprocessed_frame

# Open the video capture
cap = cv2.VideoCapture(0)  # Use 0 for the default camera or provide the appropriate index for other cameras

while True:
    # Read a frame from the video capture
    ret, frame = cap.read()

    if not ret:
        break

    # Preprocess the frame
    preprocessed_frame = preprocess_frame(frame)

    # Perform inference
    prediction = model.predict(preprocessed_frame)
    predicted_class = int(np.round(prediction[0]))

    # Get the predicted class label
    class_label = class_labels[predicted_class]

    # Draw the bounding box and class label on the frame
    cv2.putText(frame, class_label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    cv2.imshow('Face Mask Detection', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close the OpenCV windows
cap.release()
cv2.destroyAllWindows()